In [2202]:
import numpy as np
import sympy as sp
import pandas as pd
import os
from sympy.parsing.sympy_parser import parse_expr
import control

In [2203]:
# Parameters
r, Mb, rb, Ib, Mw, Iw, g, tau = sp.symbols('r M_b r_b I_b M_w I_w g tau')

In [2204]:
subs_dict = {
    Mb: 0.366,    # Body Mass
    rb: 0.09,    # Pendulum center of mass
    Ib: 0.0082,   # Body Inertia
    Mw: 0.409 * 2,    # Wheel mass
    Iw: 0.00029,   # Wheel inertia
    r: 0.06,    # Wheel radius
    g: 9.81     # Gravity
}

# Linear Controller Design

State Space Function(Continuous and Discrete)

In [2205]:
# Import linearied A and B
with open('../dynamics/output/A.txt', 'r') as f:
    A = parse_expr(f.read())

with open('../dynamics/output/B.txt', 'r') as f:
    B = parse_expr(f.read())

# print(A)
# print(B)

In [2206]:
A_num = np.array(A.subs(subs_dict).evalf().tolist()).astype(np.float64)
B_num = np.array(B.subs(subs_dict).evalf().tolist()).astype(np.float64)
C_num = np.array([
    [1, 0, 0, 0],   # x from encoder
    [0, 1, 0, 0],   # ẋ
    [0, 0, 1, 0],   # θ
    [0, 0, 0, 1]    # θ̇
])
D_num = np.zeros((4, 1))

sys = control.ss(A_num, B_num, C_num, D_num)

In [2207]:
# discrete system initialization
dt = 0.005
sys_d = control.c2d(sys, dt)

In [2208]:
os.makedirs('data/statespace_models', exist_ok=True)

# --- Save continuous system as CSV ---
pd.DataFrame(sys.A).to_csv('data/statespace_models/A_continuous.csv', index=False, header=False)
pd.DataFrame(sys.B).to_csv('data/statespace_models/B_continuous.csv', index=False, header=False)
pd.DataFrame(sys.C).to_csv('data/statespace_models/C_continuous.csv', index=False, header=False)
pd.DataFrame(sys.D).to_csv('data/statespace_models/D_continuous.csv', index=False, header=False)

# --- Save discrete system as CSV ---
pd.DataFrame(sys_d.A).to_csv('data/statespace_models/A_discrete.csv', index=False, header=False)
pd.DataFrame(sys_d.B).to_csv('data/statespace_models/B_discrete.csv', index=False, header=False)
pd.DataFrame(sys_d.C).to_csv('data/statespace_models/C_discrete.csv', index=False, header=False)
pd.DataFrame(sys_d.D).to_csv('data/statespace_models/D_discrete.csv', index=False, header=False)

# Save dt (time step) as simple text file
with open('data/statespace_models/dt_discrete.txt', 'w') as f:
    f.write(str(sys_d.dt))


## Controllability and Observability

In [2209]:
ctrb_matrix = control.ctrb(A_num, B_num)
print("Rank of ctrb:", np.linalg.matrix_rank(ctrb_matrix))

Rank of ctrb: 4


In [2210]:
eigvals, eigvecs = np.linalg.eig(A_num)
# for i, (eigval, eigvec) in enumerate(zip(eigvals, eigvecs.T)):
#     print(f"Eigenvalue {eigval:.4f}:")
#     print(f"Corresponding eigenvector:\n{eigvec}\n")

In [2211]:
O = control.obsv(sys.A, sys.C) 
print(np.linalg.matrix_rank(O))

4


## LQR

In [2212]:
Q = np.array([
    [1, 0, 0, 0],
    [0, 5, 0, 0],
    [0, 0, 10, 0],
    [0, 0, 0, 10]
])
R = np.array([[3]])
K, S, E = control.lqr(sys.A, sys.B, Q, R)
print("LQR gain K:", K)
print("LQR gain E:", E)

LQR gain K: [[0.57735027 1.91360849 5.35670085 2.08410568]]
LQR gain E: [-255.09460549+0.j           -0.45613351+0.j
   -1.14637831+0.85802351j   -1.14637831-0.85802351j]


In [2213]:
# Ensure the folder exists
os.makedirs('data/gain_matrices', exist_ok=True)

# Save the K matrix
pd.DataFrame(K).to_csv('data/gain_matrices/K_Continuous.csv', index=False, header=False)

In [2214]:
# A_cl = A_num - B_num @ K
# eigvals, eigvecs = np.linalg.eig(A_cl)
# for i, (eigval, eigvec) in enumerate(zip(eigvals, eigvecs.T)):
#     print(f"Eigenvalue {eigval:.4f}:")
#     print(f"Corresponding eigenvector:\n{eigvec}\n")
# sys_cl = control.ss(A_cl, B_num, C_num, D_num)

In [2215]:
# # Gramian
# Wc = solve_continuous_lyapunov(A_cl, -B_num @ B_num.T)
# print("Controllability Gramian (via SciPy):\n", Wc)


## Discrete LQR

In [2216]:
Q_d = np.array([
    [0.1, 0, 0, 0],
    [0, 1000, 0, 0],
    [0, 0, 10, 0],
    [0, 0, 0, 1]
])
R_d = np.array([[1]])
K_d, S_d, E_d = control.dlqr(sys_d.A, sys_d.B, Q_d, R_d)
print("LQR gain K:", K_d)
print("LQR gain E:", E_d)

LQR gain K: [[ 0.10045949 10.08750993 12.59321803  2.54164549]]
LQR gain E: [0.10594577+0.j         0.99995   +0.j         0.97602013+0.00232763j
 0.97602013-0.00232763j]


In [2217]:
# Ensure the folder exists
os.makedirs('data/gain_matrices', exist_ok=True)

# Save the K matrix
pd.DataFrame(K_d).to_csv('data/gain_matrices/K_Discrete.csv', index=False, header=False)

## LQG

In [2218]:
x_std = 0.01      # encoder error in meters
dx_std = 0.2       # IMU-integrated linear velocity (m/s)
theta_std = 0.1    # integrated angle error (rad)
dtheta_std = 0.1  # IMU gyro noise (rad/s)

# Rn = np.diag([1e-6, 1e-6, 1e-6, 1e-6])  # reduce measurement noise → trust sensors
Rn = np.diag([
    (x_std),
    (dx_std),
    (theta_std),
    (dtheta_std)
])
G = np.eye(4)
# Qn = np.diag([1e6, 1e6, 1e6, 1e6])      # increase process noise → fast observer
Qn = np.diag([
    1,   # x
    1,   # dx (e.g., friction force uncertainty)
    1,   # theta (e.g., small external torque)
    1    # dtheta (e.g., fast unmodeled torque effect)
])
L, P, E = control.lqe(sys.A, G, sys.C, Qn, Rn)
print(L)

[[ 1.03058573e+01  1.69723476e-01 -3.32523450e-03 -1.41885111e-02]
 [ 3.39446952e+00  2.10808302e+00 -9.71695498e-02 -3.30159358e-01]
 [-3.32523450e-02 -4.85847749e-02  1.62630956e+00  3.88968609e+00]
 [-1.41885111e-01 -1.65079679e-01  3.88968609e+00  1.54505803e+01]]


In [2219]:
# Ensure the folder exists
os.makedirs('data/gain_matrices', exist_ok=True)

# Save the L matrix
pd.DataFrame(L).to_csv('data/gain_matrices/L_Continuous.csv', index=False, header=False)

## Discrete LQG

In [2220]:
x_std_d = 0.01      # encoder error in meters
dx_std_d = 0.2       # IMU-integrated linear velocity (m/s)
theta_std_d = 0.1    # integrated angle error (rad)
dtheta_std_d = 0.1  # IMU gyro noise (rad/s)


# Rn = np.diag([1e-6, 1e-6, 1e-6, 1e-6])  # reduce measurement noise → trust sensors
Rn_d = np.diag([
    (x_std_d),
    (dx_std_d),
    (theta_std_d),
    (dtheta_std_d)
])
G_d = np.eye(4)
# Qn = np.diag([1e6, 1e6, 1e6, 1e6])      # increase process noise → fast observer
Qn_d = np.diag([
    100,   # x
    100,   # dx (e.g., friction force uncertainty)
    100,   # theta (e.g., small external torque)
    100    # dtheta (e.g., fast unmodeled torque effect)
])
L_d, P_d, E_d = control.dlqe(sys_d.A, G_d, sys_d.C, Qn, Rn)
print(L_d)
print(E_d)

[[ 9.90195784e-01  4.27662822e-03 -9.59008174e-06 -6.18461535e-08]
 [ 1.22355400e-04  8.54102140e-01 -3.78760149e-03 -2.05696298e-05]
 [-5.94411280e-08 -2.31785052e-05  9.16436154e-01  5.63177170e-03]
 [-1.82769923e-08 -7.09565789e-06  1.44676743e-01  9.16757042e-01]]
[0.00980487+0.j         0.1458971 +0.j         0.08379539+0.00275977j
 0.08379539-0.00275977j]


In [2221]:
# Ensure the folder exists
os.makedirs('data/gain_matrices', exist_ok=True)

# Save the L matrix
pd.DataFrame(L_d).to_csv('data/gain_matrices/L_Discrete.csv', index=False, header=False)

# Nonlinear Controller Design

In [2222]:
with open('../dynamics/output/A_func.txt', 'r') as f:
    A_func = parse_expr(f.read())
A_func_num = A_func.subs(subs_dict)
display(A_func_num)
# print(A_func_num)

Matrix([
[0, 1,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              0,                                                                                                                    0],
[0, 0,                   0.358623393996421*tau*sin(theta(t))*cos(

In [2223]:
with open('../dynamics/output/xdd_solution.txt', 'r') as f:
    xdd_solution = parse_expr(f.read())

with open('../dynamics/output/thetadd_solution.txt', 'r') as f:
    thetadd_solution = parse_expr(f.read())

xdd_partial = xdd_solution.subs(subs_dict)
thetadd_partial = thetadd_solution.subs(subs_dict)
display(xdd_partial)
display(thetadd_partial)

-0.000118584*tau*cos(theta(t))/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) - 0.000669876*tau/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) - 3.83193997776e-5*sin(theta(t))*cos(theta(t))/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) + 1.3239429264e-6*sin(theta(t))*Derivative(theta(t), t)**2/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2)

0.0019764*tau*cos(theta(t))/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) + 0.0045524*tau/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) - 3.90615696e-6*sin(theta(t))*cos(theta(t))*Derivative(theta(t), t)**2/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) + 0.00147106890936*sin(theta(t))/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2)

## Extended Kalman Filter

<img src="Diagrams/EKF/EKF.png" alt="Alt text" width="600" height="500"/>

In [2224]:
Qn_nonlinear = np.diag([
    1,   # x
    1,   # dx (e.g., friction force uncertainty)
    1,   # theta (e.g., small external torque)
    1    # dtheta (e.g., fast unmodeled torque effect)
])

In [2225]:
x_std_nonlinear = 0.01      # encoder error in meters
dx_std_nonlinear = 0.2       # IMU-integrated linear velocity (m/s)
theta_std_nonlinear = 0.1    # integrated angle error (rad)
dtheta_std_nonlinear = 0.1  # IMU gyro noise (rad/s)


# Rn = np.diag([1e-6, 1e-6, 1e-6, 1e-6])  # reduce measurement noise → trust sensors
Rn_nonlinear = np.diag([
    (1e-3),
    (1e-3),
    (1e-4),
    (1e-4)
])

In [2226]:
# Ensure the folder exists
os.makedirs('data/nonlinear_matrices', exist_ok=True)

# Save the Qn matrix
pd.DataFrame(Qn_nonlinear).to_csv('data/nonlinear_matrices/Qn.csv', index=False, header=False)
# Save the Rn matrix
pd.DataFrame(Rn_nonlinear).to_csv('data/nonlinear_matrices/Rn.csv', index=False, header=False)